# Toying with my Algorithm

In [1]:
from splitwise import OptimalSplit
from collections import defaultdict
from math import inf, isclose

In [25]:
transactions = [["Angela", "Olaf", 10],
    ["Olaf", "Friedrich", 5],
    ["Friedrich", "Angela", 5],
    ["Friedrich", "Barbie", 10],
    ["Barbie", "Olaf", 7],
    ["Ken", "Barbie", 5],
    ["Barbie", "Ken", 5]]

splitter = OptimalSplit()
txns = splitter.minTransfers(transactions)

print(f"{len(txns)} payments need to be made.")

for debtor, creditor, amount in txns:
    print(f"{debtor} pays {creditor} {amount}")






defaultdict(<class 'int'>, {'Angela': 5, 'Olaf': -12, 'Friedrich': 10, 'Barbie': -3, 'Ken': 0})
{'Angela': 5, 'Olaf': -12, 'Friedrich': 10, 'Barbie': -3}
3 payments need to be made.
Olaf pays Angela 5
Olaf pays Friedrich 7
Barbie pays Friedrich 3


In [27]:
transactions = [["Jasmin", "Hanna", 10],
    ["Jasmin", "Carl", 6.66]]

splitter = OptimalSplit()
txns = splitter.minTransfers(transactions)

print(f"{len(txns)} payments need to be made.")

for debtor, creditor, amount in txns:
    print(f"{debtor} pays {creditor} {amount}")

defaultdict(<class 'int'>, {'Jasmin': 16.66, 'Hanna': -10, 'Carl': -6.66})
{'Jasmin': 16.66, 'Hanna': -10, 'Carl': -6.66}
2 payments need to be made.
Hanna pays Jasmin 10
Carl pays Jasmin 6.66


In [12]:
float(True)

1.0

In [ ]:
class ExperimentalSplit:
    """
    Core algorithm for debt splitting with minimal amount of payments.
    """
    def expTransfers(self, transactions):
        """
        Performs debt split with minimal amount of payments.

        Args:
        - transactions: a list of [lender, borrower, amount] entries.

        Returns:
        - A list of transactions of the form [creditor, debtor, amount].
        """
        score = defaultdict(int) # default value of every new key in the score-dict is set to 0 

        #creates score-dict. Keys: names of group members, Values: total amount owed/borrowed.
        for lender, borrower, amount in transactions:
            score[lender] += amount
            score[borrower] -= amount


         # for us to check what's going on 
        print(score)


        # remove settled accounts (accounts with balance = 0)
        debt = {person: amt for person, amt in score.items() if amt != 0}
        people = list(debt.keys())
        balances = list(debt.values())

        #to check that all accounts with balance 0 have been removed 
        print(debt)

        def dfs(start, balances, people):
            # skip over people with balance == 0 (settled debts)
            while start < len(balances) and abs(balances[start]) < 1e-9: # floating point safeguard 
                start += 1
            # base case: there are no balances to settle --> return 0 (0 payments need to be made) and empty list of transactions    
            if start == len(balances):
                return 0, []
            #initialize minimum number of payments and optimal transaction list
            min_payments = inf
            best_path = []

            # attempt debt settling by pairing balances[start] with each of the later people in the list 
            for i in range(start + 1, len(balances)):
                if balances[start] * balances[i] < 0:# ensures that only debtors & creditors can be paired, not debtor & debtor or creditor & creditor
                    amount = min(abs(balances[start]), abs(balances[i]))#amount to be paid in the simulated transaction 
                    #save balance-values for backtracing
                    start_original = balances[start]
                    i_original = balances[i]

                    # This block is mainly helpful for debugging with print-statements.
                    # It determine direction of payment: from start to i or from i to start. 
                    # It also simulates the payments for us to keep track of them. 
                    if balances[start] < 0: #Case 1: if `start` owes money
                        path = [(people[start], people[i], amount)]
                        #reflect the changes of the transactions in the balances list 
                        balances[i] -= amount
                        balances[start] += amount
                    else:  #Case 2: if `start` gets money
                        path = [(people[i], people[start], amount)]
                        balances[start] -= amount
                        balances[i] += amount

                    # determine whether to move to the next person or stay with 'start' depending on whether `start` has been settled
                    if isclose(balances[start], 0, abs_tol= 1e-6): # Case 1: `start` is settled and we move on 
                        next_start = start + 1
                    else:
                        next_start = start # Case 2: `start` is not settled and we stay with it 

                    # recursive settlement of remaining debt 
                    payments, next_path = dfs(next_start, balances[:], people)
                    if payments + 1 < min_payments: # if path results in fewer payments than before, update best solution 
                        min_payments = payments + 1
                        best_path = path + next_path

                    # restore original balances for next depth-first search
                    balances[start] = start_original
                    balances[i] = i_original
                
            return min_payments, best_path


        # run recursive function starting from first unsettled person     
        _, result = dfs(0, balances, people)
        
        return result




In [9]:
transactions = [["Jasmin", "Hanna", True],
    ["Jasmin", "Carl", 6.66]]

splitter = ExperimentalSplit()
txns = splitter.expTransfers(transactions)

print(f"{len(txns)} payments need to be made.")

for debtor, creditor, amount in txns:
    print(f"{debtor} pays {creditor} {amount}")

IndexError: list index out of range